In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import copy
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import librosa
import matplotlib.pyplot as plt

os.chdir(path) ## can change current directory    np.argmax(x)

In [ ]:
# np.argmax(x)
def find_max_and_return_index(mylist):
    co = 0
    mymax = 0
    for i,value in enumerate(mylist):
        if mymax < value:
            mymax = value
            co = i
    return mymax,co

In [ ]:
list_folder = os.listdir('/kaggle/input/gunshot-audio-dataset/')

hop_length = 512 #the default spacing between frames
n_fft = 255 #number of samples

raw_x = []
raw_y = []
for i in list_folder:
    list_file_wave = os.listdir('/kaggle/input/gunshot-audio-dataset/' + i)
    for j in list_file_wave:
        filename = '/kaggle/input/gunshot-audio-dataset/' + i + '/' + j
        data, sr = librosa.load(filename,sr=22050)
        
        raw_x.append(data)
        raw_y.append(i)
        
        
   

In [ ]:
x = []
y = []
for i in range(len(raw_x)):
    data = raw_x[i]
    if len(data) == 44100:
        mfcc_data = np.array(librosa.feature.mfcc(data, n_fft=n_fft,hop_length=hop_length,n_mfcc=128))
        x.append(mfcc_data)
        y.append(raw_y[i])

In [ ]:
x_copy = copy.copy(x)
y_copy = copy.copy(y)

In [ ]:
#x_copy = np.array((x_copy-np.min(x_copy))/(np.max(x_copy)-np.min(x_copy)))
#x_copy = x_copy/np.std(x_copy)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y_copy)
y_copy_new = le.transform(y_copy)

# from tensorflow.keras.utils import to_categorical
# y_copy_new = to_categorical(y_copy_new, dtype ="uint8")

In [ ]:
len_x_copy = []
for i in range(len(x_copy)):
    temp = x_copy[i].shape
    # print(temp)
    len_x_copy.append(temp)

In [ ]:
value, count = np.unique(len_x_copy, return_counts=True)
print(value,count)

In [ ]:
x_copy = np.array(x_copy)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_copy, y_copy_new, test_size=0.25, random_state=123, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=123)


In [ ]:
from tensorflow.keras.layers import LSTM, Dense,Dropout,Flatten
from tensorflow import keras

In [ ]:

input_shape = x_copy[0].shape

model = keras.Sequential()
model.add(LSTM(128,input_shape=input_shape,return_sequences=True))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(9, activation='softmax'))
model.summary()
model.compile(loss="SparseCategoricalCrossentropy", optimizer="adam", metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=72, validation_data=(X_val, y_val), shuffle=False)

remember2 = sorted(remember,reverse=True)

In [ ]:
#Adapted from Deep Learning with Python by Francois Chollet, 2018
history_dict=history.history
loss_values=history_dict['loss']
acc_values=history_dict['accuracy']
val_loss_values = history_dict['val_loss']
val_acc_values=history_dict['val_accuracy']
epochs=range(1,51)
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
ax1.plot(epochs,loss_values,'co',label='Training Loss')
ax1.plot(epochs,val_loss_values,'m', label='Validation Loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax2.plot(epochs,acc_values,'co', label='Training accuracy')
ax2.plot(epochs,val_acc_values,'m',label='Validation accuracy')
ax2.set_title('Training and validation accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()
plt.show()

In [ ]:
from tensorflow.math import confusion_matrix

TrainLoss, Trainacc = model.evaluate(X_train,y_train)
TestLoss, Testacc = model.evaluate(X_test, y_test)
y_pred=model.predict(X_test)
print('Confusion_matrix: ',confusion_matrix(y_test, np.argmax(y_pred,axis=1)))

In [ ]:
import librosa.display
fig, ax = plt.subplots(figsize=(20,7))
librosa.display.specshow(X_train[0],sr=sr, cmap='cool',hop_length=hop_length)
ax.set_xlabel('Time', fontsize=15)
ax.set_title('MFCC', size=20)
plt.colorbar()
plt.show()